In [1]:
#Data
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import shutil

#Graph Counstruction
import torch
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch.nn import Linear
from torch.nn import ReLU
import torch.nn as nn
from torch.nn.init import kaiming_uniform_

In [ ]:
train_path = 'io/input/data_raw/train.csv'
test_path = 'io/input/data_raw/example_test.csv'
assets_details_path ='io/input/data_raw/asset_details.csv'
data_intermediate_path ='io/input/data_intermediate/'
train_obj_path = data_intermediate_path + 'train_object.pkl'
train_node_features_path = data_intermediate_path + 'node_features/train/'
train_node_labels_path = data_intermediate_path + 'node_labels/train/'
test_node_labels_path = data_intermediate_path + 'node_labels/test/'
test_node_features_path = data_intermediate_path + 'node_features/test/'
chunk_size = 100000
timesteps = 1200

In [ ]:
def load_object(path):
    num_chunks = len([f for f in os.listdir(path) if f.startswith('chunk_') and f.endswith('.npy')])
    # Load array from chunks
    chunks = []
    for i in range(num_chunks):
        filename = f"{path}chunk_{i}.npy"
        print()
        chunk = np.load(filename,allow_pickle=True)
        chunks.append(chunk)
    arr_reconstructed = np.concatenate(chunks, axis=0)
    return arr_reconstructed

In [ ]:
train_node_features = load_object(train_node_features_path)
train_node_labels = load_object(train_node_labels_path)
#test_node_features = load_object(test_node_features_path)
edges = np.array([[0,1,2,3,4,5,6,7,8,9,10,11,12,13,11],[1,0,1,8, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2]])
edges_weights = np.array([1,0.1,1,0.2,0.5,0.1,0,1,0.2,0.5,0.1,1,0.1,1,0.2]).T
print('Train Node Features Train', train_node_features.shape)
#print(test_node_features.shape)
print('Train Node Labels Train', train_node_labels.shape)
print('Edges Shape', edges.shape)
print('Edges Weights Shapes',edges_weights.shape)

In [ ]:
graTrain=StaticGraphTemporalSignal(edge_index=edges,edge_weight=edges_weights,
                                   features=train_node_features,targets=train_node_labels)

In [ ]:
train_dataset = graTrain
#train_input = np.array(train_dataset.features)

In [ ]:
train_dataset.features.shape

In [ ]:
batch_size=64
train_dataset = graTrain
train_input = np.array(train_dataset.features) # (5074, 222, 24, 1)
train_target = np.array(train_dataset.targets) # (5074, 207, 1)
train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor)  # (B, N, F, T)
train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor)  # (B, N, T)
train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size,drop_last=True)

In [ ]:
for snapshot in train_dataset:
    static_edge_index = snapshot.edge_index
    print(snapshot)
    break;

In [ ]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.tgnn = A3TGCN2(in_channels=node_features,  out_channels=32, periods=periods,batch_size=batch_size) # node_features=2, periods=12
        # Equals single-shot prediction
        
        
        self.linear = torch.nn.Linear(32, periods)

    def forward(self, x, edge_index):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index) # x [b, 207, 2, 12]  returns h [b, 207, 12]
        h = F.relu(h) 
        h = self.linear(h)
        return h

In [ ]:
# Create model and optimizers
model = TemporalGNN(node_features=9, periods=1, batch_size=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()


# print('Net\'s state_dict:')
# total_param = 0
# for param_tensor in model.state_dict():
#     print(param_tensor, '\t', model.state_dict()[param_tensor].size())
#     total_param += np.prod(model.state_dict()[param_tensor].size())
# print('Net\'s total params:', total_param)
# #--------------------------------------------------
# print('Optimizer\'s state_dict:')
# for var_name in optimizer.state_dict():
#     print(var_name, '\t', optimizer.state_dict()[var_name])

In [ ]:
model.train()

for epoch in range(10):
    step = 0
    loss_list = []
    for encoder_inputs, labels in train_loader:
        y_hat = model(encoder_inputs, static_edge_index)         # Get model predictions
        loss = loss_fn(y_hat, labels) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step= step+ 1
        loss_list.append(loss.item())
        if step % 100 == 0 :
            print(sum(loss_list)/len(loss_list))
    print("Epoch {} train RMSE: {:.4f}".format(epoch, sum(loss_list)/len(loss_list)))